<a href="https://colab.research.google.com/github/SophieShin/MachineLearning/blob/main/Option_Pricing(%2Bprc_lmt)_MondayOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 반드시 사본 저장 후 사본파일에서 실행해주세요.(파일/Drive에 사본저장)
#### 아래 '재생'버튼을 누르면서 진행하면 됩니다


In [3]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np

#### [CR20]화면에서 검증하고자 하는 상품의 이론가격 파라미터를 저장합니다
#### 결제월 : 전체
#### 선물옵션구분: 콜+풋
#### 이론가용도: 기준가용
#### 화면에서 프린트 버튼을 누른 뒤 엑셀로 저장하면 됩니다.
#### 엑셀로 저장할 때  'CSV' 형식으로 저장합니다.
#### 엑셀로 저장할 때 파라미터를 제외한 문구들(맨 위에 화면구성, 맨 아래에 주석 등)을 지우고 작업하는게 편합니다

In [4]:
# 파일 업로드
# 예제파일 2개(이론가파람_SK하이닉스, 상하한가) 파일을 https://github.com/SophieShin/MachineLearning/tree/main 에서 다운 받을 수 있습니다.

from google.colab import files

uploaded = files.upload()

Saving 월요일옵션.csv to 월요일옵션.csv


#### 아래 uploaded['이론가파람_SK하이닉스.csv'] 부분에서 한글 부분을 자신이 저장한 파일 이름으로 변경합니다

In [5]:
import io
df = pd.read_csv(io.BytesIO(uploaded['월요일옵션.csv']), encoding='utf-8', thousands = ',')
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,C 2307W4 332.5,콜,2023-07-25,332.5,371.54,9,3.75,0.0,0.275,39.37
1,C 2307W4 335.0,콜,2023-07-25,335.0,371.54,9,3.75,0.0,0.275,36.89
2,C 2307W4 337.5,콜,2023-07-25,337.5,371.54,9,3.75,0.0,0.275,34.41
3,C 2307W4 340.0,콜,2023-07-25,340.0,371.54,9,3.75,0.0,0.275,31.96
4,C 2307W4 342.5,콜,2023-07-25,342.5,371.54,9,3.75,0.0,0.275,29.52
...,...,...,...,...,...,...,...,...,...,...
61,P 2307W4 402.5,풋,2023-07-25,402.5,371.54,9,3.75,0.0,0.030,30.59
62,P 2307W4 405.0,풋,2023-07-25,405.0,371.54,9,3.75,0.0,0.030,33.09
63,P 2307W4 407.5,풋,2023-07-25,407.5,371.54,9,3.75,0.0,0.030,35.58
64,P 2307W4 410.0,풋,2023-07-25,410.0,371.54,9,3.75,0.0,0.030,38.08


In [6]:
# null 값 제거
df= df.dropna(subset=['종목명'])

In [7]:
# 금리 단위 맞추기
df['금리(%)']= df['금리(%)']/100

In [8]:
# 풋옵션 이론가 산출 파라미터만 남기기
df2 = df[df['선물옵션구분'] == '풋']

In [9]:
# 콜옵션 이론가 산출 파라미터만 남기기
df = df[df['선물옵션구분'] == '콜']

#### 1. 콜옵션 이론가 구하기

In [10]:
# null값 제거
df = df.dropna(axis = 1)
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,C 2307W4 332.5,콜,2023-07-25,332.5,371.54,9,0.0375,0.0,0.275,39.37
1,C 2307W4 335.0,콜,2023-07-25,335.0,371.54,9,0.0375,0.0,0.275,36.89
2,C 2307W4 337.5,콜,2023-07-25,337.5,371.54,9,0.0375,0.0,0.275,34.41
3,C 2307W4 340.0,콜,2023-07-25,340.0,371.54,9,0.0375,0.0,0.275,31.96
4,C 2307W4 342.5,콜,2023-07-25,342.5,371.54,9,0.0375,0.0,0.275,29.52
5,C 2307W4 345.0,콜,2023-07-25,345.0,371.54,9,0.0375,0.0,0.275,27.12
6,C 2307W4 347.5,콜,2023-07-25,347.5,371.54,9,0.0375,0.0,0.275,24.74
7,C 2307W4 350.0,콜,2023-07-25,350.0,371.54,9,0.0375,0.0,0.275,22.43
8,C 2307W4 352.5,콜,2023-07-25,352.5,371.54,9,0.0375,0.0,0.275,20.17
9,C 2307W4 355.0,콜,2023-07-25,355.0,371.54,9,0.0375,0.0,0.275,17.99


In [11]:
# 이론가격을 구하기 위해 data 자료타입 변경

S = list(map(float,df['기초자산\n기준가격'].tolist()))
K = list(map(float,df['행사가격'].tolist()))
r = list(map (float,df['금리(%)'].tolist()))
div = list(map(float,df['배당  현재가치'].tolist()))
sigma = list(map(float,df['변동성'].tolist()))
t = list(map(int,df['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df['이론가격'].tolist()))
name = df['종목명'].tolist()

In [13]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\n기준가격']*1.2
lower_lmt =  df['기초자산\n기준가격']*0.8
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [14]:
# call 옵션 이론가격 구하는 코드

def call(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    C_i = max((s-Div) * u**i * d**(49-i) -k , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(C_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [15]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드
call_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  call(s,k,R,T,Div, Sigma)
  call_price_temp.append(price)


In [16]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  call(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)


In [17]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = call(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [18]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(call_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df['theo_price_검증용'] = call_price
df['시스템 검증 차이'] = df['이론가격'] - df['theo_price_검증용']
df['상한가'] = upper_limit
df['하한가'] = lower_limit

In [19]:
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,C 2307W4 332.5,콜,2023-07-25,332.5,371.54,9,0.0375,0.0,0.275,39.37,39.368757,0.001243,112.809654,0.0
1,C 2307W4 335.0,콜,2023-07-25,335.0,371.54,9,0.0375,0.0,0.275,36.89,36.885490,0.004510,110.312625,0.0
2,C 2307W4 337.5,콜,2023-07-25,337.5,371.54,9,0.0375,0.0,0.275,34.41,34.414406,-0.004406,107.816862,0.0
3,C 2307W4 340.0,콜,2023-07-25,340.0,371.54,9,0.0375,0.0,0.275,31.96,31.958292,0.001708,105.321130,0.0
4,C 2307W4 342.5,콜,2023-07-25,342.5,371.54,9,0.0375,0.0,0.275,29.52,29.516055,0.003945,102.825397,0.0
5,C 2307W4 345.0,콜,2023-07-25,345.0,371.54,9,0.0375,0.0,0.275,27.12,27.116741,0.003259,100.331738,0.0
6,C 2307W4 347.5,콜,2023-07-25,347.5,371.54,9,0.0375,0.0,0.275,24.74,24.737017,0.002983,97.839306,0.0
7,C 2307W4 350.0,콜,2023-07-25,350.0,371.54,9,0.0375,0.0,0.275,22.43,22.429725,0.000275,95.346874,0.0
8,C 2307W4 352.5,콜,2023-07-25,352.5,371.54,9,0.0375,0.0,0.275,20.17,20.173926,-0.003926,92.855943,0.0
9,C 2307W4 355.0,콜,2023-07-25,355.0,371.54,9,0.0375,0.0,0.275,17.99,17.992125,-0.002125,90.371141,0.0


In [20]:
df.to_excel('미니옵션콜_검증용.xlsx')

In [21]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df[df['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


#### 2. 풋옵션 이론가 구하기

In [22]:
# null값 제거
df2 = df2.dropna(axis = 1)
df2

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
33,P 2307W4 332.5,풋,2023-07-25,332.5,371.54,9,0.0375,0.0,0.03,0.00
34,P 2307W4 335.0,풋,2023-07-25,335.0,371.54,9,0.0375,0.0,0.03,0.00
35,P 2307W4 337.5,풋,2023-07-25,337.5,371.54,9,0.0375,0.0,0.03,0.00
36,P 2307W4 340.0,풋,2023-07-25,340.0,371.54,9,0.0375,0.0,0.03,0.00
37,P 2307W4 342.5,풋,2023-07-25,342.5,371.54,9,0.0375,0.0,0.03,0.00
38,P 2307W4 345.0,풋,2023-07-25,345.0,371.54,9,0.0375,0.0,0.03,0.00
39,P 2307W4 347.5,풋,2023-07-25,347.5,371.54,9,0.0375,0.0,0.03,0.00
40,P 2307W4 350.0,풋,2023-07-25,350.0,371.54,9,0.0375,0.0,0.03,0.00
41,P 2307W4 352.5,풋,2023-07-25,352.5,371.54,9,0.0375,0.0,0.03,0.00
42,P 2307W4 355.0,풋,2023-07-25,355.0,371.54,9,0.0375,0.0,0.03,0.00


In [23]:
#인덱스 초기화
df2.reset_index(inplace = True)

In [25]:
# 이론가격을 구하기 위해 data 자료구조 변경

S = list(map(float,df2['기초자산\n기준가격'].tolist()))
K = list(map(float,df2['행사가격'].tolist()))
r = list(map (float,df2['금리(%)'].tolist()))
div = list(map(float,df2['배당  현재가치'].tolist()))
sigma = list(map(float,df2['변동성'].tolist()))
t = list(map(int,df2['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df2['이론가격'].tolist()))
name = df2['종목명'].tolist()

In [27]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\n기준가격']*0.8
lower_lmt =  df['기초자산\n기준가격']*1.8
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [28]:
# put 옵션 이론가격 구하는 코드

def put(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    P_i = max(k -(s-Div)* u**i * d**(49-i)  , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(P_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [29]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드

put_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  put(s,k,R,T,Div,Sigma)
  put_price_temp.append(price)


In [30]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  put(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)

In [31]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = put(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [32]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(put_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df2['theo_price_검증용'] = call_price
df2['시스템 검증 차이'] = df2['이론가격'] - df2['theo_price_검증용']
df2['상한가'] = upper_limit
df2['하한가'] = lower_limit

In [33]:
df2

,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,33,P 2307W4 332.5,풋,2023-07-25,332.5,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,35.192693,0.0
1,34,P 2307W4 335.0,풋,2023-07-25,335.0,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,37.690383,0.0
2,35,P 2307W4 337.5,풋,2023-07-25,337.5,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,40.188072,0.0
3,36,P 2307W4 340.0,풋,2023-07-25,340.0,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,42.685762,0.0
4,37,P 2307W4 342.5,풋,2023-07-25,342.5,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,45.183451,0.0
5,38,P 2307W4 345.0,풋,2023-07-25,345.0,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,47.681141,0.0
6,39,P 2307W4 347.5,풋,2023-07-25,347.5,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,50.178830,0.0
7,40,P 2307W4 350.0,풋,2023-07-25,350.0,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,52.676519,0.0
8,41,P 2307W4 352.5,풋,2023-07-25,352.5,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,55.174209,0.0
9,42,P 2307W4 355.0,풋,2023-07-25,355.0,371.54,9,0.0375,0.0,0.03,0.00,0.000000e+00,0.000000e+00,57.671898,0.0


In [34]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df2[df2['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


In [35]:
df2.to_excel('미니옵션풋_검증용.xlsx')